Author: Rory Portman <br>
Email:&nbsp;&nbsp; portmanrk@gmail.com

This notebook forms part of my BSc research project on Binding Free Energy Calculation protocols for Small molecule:RNA Complexes.

# BioSimSpace

## Molecular Dynamics Simulations and Relative Binding Free Energy Calculations Setup and Analysis for Small Molecule:RNA Complexes

This tutorial covers the necessary steps to setup, run and analyse RBFE calculations for Small Molecule:RNA complexes. Included in this tutorial:

* Loading, parameterising, solvating and merging the Ligands, the RNA and the Waters.
* Running Equilibration and Minimisation for the Small Molecule:RNA system.
* Writing scripts to initiate 100 ns molecular dynamics simulations using various simulation engines.
* Analysing MD simulations and producing plots for reports.

* Loading, parameterising, solvating and merging ligands, the RNA and the waters for RBFE calculations. Includes making merged molecules.
* Generating the RBFE input (.pert) files
* Analysing the RBFE



For this tutorial we will be using (ligands and rna from Connely 2019 etc.) "include introduction for ligands and rna here. reference and provide links"

Firstly let's import all necessary directories below:

In [1]:
#IMPORT DIRECTORIES

from get_tutorial import download
download()

import BioSimSpace as BSS
import os
import numpy as np
import matplotlib.pyplot as plt
from BioSimSpace import _Exceptions
import sys
import csv

INFO:rdkit:Enabling RDKit 2023.03.3 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


## Objective 2: Small Molecule:RNA system setup.

Before beginning RBFE calculations it is important to determine whether the solvated Small Moleucle:Rna system is stable and how the RNA flexibility affects the ligand binding site.

First we must load in the ligands (include name and shorthand name) and the RNA, and parameterise them using existing BioSimSpace Forcefields (include a link?).

- Ligand_1: 2-[(dibenzo[b,d]furan-2-yl)oxy]ethan-1-amine
- Ligand_2: 2-[(dibenzo[b,d]furan-2-yl)oxy]-N,N-dimethylethan-1-amine
- Ligand_3: 2-[(9H-carbazol-3-yl)oxy]-N,N-dimethylethan-1-amine

The RNA required the following approach to manually parameterise: 


The waters were parameterised using the following approach:


Prior to loading, ligand 1 is protonated using obabel taking the charge of the ligand into account:

* obabel -i pdb lig_1.pdb -o pdb -O lig_1_protonated.pdb -p

In [3]:
#LOAD LIGAND, FREE_LIGAND, RNA AND WATER

ligand = BSS.IO.readMolecules("input_rory/lig_1_protonated.pdb")[0]
free_ligand = ligand
water=BSS.IO.readMolecules("input_rory/waters.*")
rna_bases = BSS.IO.readMolecules("input_rory/input_rory_3q50/rna.*")

print ("ligand =", ligand)
print("free_ligand =",free_ligand)
print("rna = ",rna)
print("water =",water)
print ("rna_bases =", rna_bases)

ligand = <BioSimSpace.Molecule: number=2, nAtoms=31, nResidues=1>
free_ligand = <BioSimSpace.Molecule: number=2, nAtoms=31, nResidues=1>
rna =  <BioSimSpace.System: nMolecules=1>
water = <BioSimSpace.System: nMolecules=83>
rna_bases = <BioSimSpace.System: nMolecules=2>



To parameterise the ligand we will use the GAFF2 amber forcefield:

In [4]:
#PARAMATERISE LIGAND, FREE_LIGAND AND WATER

gaff_free_ligand = BSS.Parameters.gaff2(free_ligand).getMolecule()
gaff_ligand = BSS.Parameters.gaff2(ligand).getMolecule()

Now we combine the parameterised RNA, ligand and waters into one system called `molecules`.

In [5]:
#COMBINE RNA, WATER AND LIGAND

molecules= rna_bases + gaff_ligand + water
view=BSS.Notebook.View(rna_bases)
view.system()

ThemeManager()

NGLWidget(gui_style='ngl')

It is useful to check our system using `BSS.Notebook.View`.

In [19]:
view_2= BSS.Notebook.View("input_rory/lig_protonated.pdb")
view_2.system()

NGLWidget(gui_style='ngl')

After creating a Small Molecule:RNA system we must solvate using `BSS.Solvent`. In this case we will use the `tip3p` solvating model and we will automatically calculate the necessary solvent box size using `.getAxisAlignedBoundingBox()`. It is important to specify the counter ion concentration.

In [ ]:
#DETERMINE BOX SIZE AND SOLVATE COMBINED SYSTEM (CHECK COUNTER IONS)

box_min, box_max = rna.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
padding = 15 * BSS.Units.Length.angstrom
box_length = max(box_size) + 2 * padding

molecules_sol = BSS.Solvent.tip3p(molecule= molecules, box=3 * [box_length], ion_conc = 0.15)
search = molecules_sol.search("not mols with atomidx 2")
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")

And the same for the free ligand.

In [ ]:
#DETERMINE BOX SIZE AND SOLVATE FREE LIGAND

free_box_min, free_box_max = free_ligand.getAxisAlignedBoundingBox()
free_box_size = [y - x for x, y in zip(free_box_min, free_box_max)]
free_padding = 15 * BSS.Units.Length.angstrom
free_box_length = max(free_box_size) + 2 * free_padding
print(free_box_length)

free_molecules_sol = BSS.Solvent.tip3p(molecule= gaff_free_ligand, box=3 * [free_box_length], ion_conc = 0.15)
search = free_molecules_sol.search("not mols with atomidx 2")
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")

## Objective 3a: Minimisation and Equilibration of Small Molecule:RNA system.

Now that we have a solvated Small Molecule:RNA system we need to ensure that the starting structure is at a local energy minimum prior to beginning the 100 ns MD simulations. 

To do this we will perform equilibration and minimisation of the ligand_1:RNA system.

The following approach for minimisation and equilibration is adapted from: https://github.com/michellab/BioSimSpaceTutorials/blob/69e2a9f97b2d2f5ae21c5c5b45a2992b2879621c/03_fep/execution_model/scripts/BSSligprep.py#L225 



In [8]:

### Settings.
minim_steps = 250
runtime_short_nvt = 5 # ps
runtime_nvt = 50 # ps RESET to 50 after TESTING ! 
runtime_npt = 200 # ps RESET to 200 after TESTING !

### preamble. tmp_dir should at some point be derived using os.environ.get("")

tmp_dir = "temp"

amber_home = "/usr/local/amber22"
pmemd_path = amber_home + "/bin/pmemd.cuda" 

In [9]:
#################
lig_p_solvated = free_molecules_sol
system_solvated = molecules_sol
BSS.IO.saveMolecules(f"{tmp_dir}/free_lig_s", lig_p_solvated, ["PRM7", "RST7"])
BSS.IO.saveMolecules(f"{tmp_dir}/mol_sys_s", system_solvated, ["PRM7", "RST7"])
### Minimise and equilibrate our systems to make the ligand relax inside the pocket.

lig_p_solvated = BSS.IO.readMolecules([f"{tmp_dir}/free_lig_s.prm7", f"{tmp_dir}/free_lig_s.rst7"])
system_solvated = BSS.IO.readMolecules([f"{tmp_dir}/mol_sys_s.prm7", f"{tmp_dir}/mol_sys_s.rst7"])

print ("lig_p_solvated is =", lig_p_solvated)
print ("original is =", free_molecules_sol)


lig_p_solvated is = <BioSimSpace.System: nMolecules=2154>
original is = <BioSimSpace.System: nMolecules=2154>


In [10]:

def runProcess(system, protocol, pmemd=False):
        """
        Given a solvated system (BSS object) and BSS protocol, run a process workflow with either 
        Sander (CPU) or pmemd.cuda (GPU). NPT is typically done with GPU to save computing time.
        Returns the processed system.
        """
        # Create the process passing a working directory.
        if not pmemd:
            process = BSS.Process.Amber(system, protocol)
        elif pmemd:
            process = BSS.Process.Amber(system, protocol, exe=pmemd_path)

        # Start the process.
        process.start()

        # Wait for the process to exit.
        process.wait()

        # Check for errors.
        if process.isError():
            print(process.stdout())
            print(process.stderr())
            raise _Exceptions.ThirdPartyError("The process exited with an error!")

        # If it worked, try to get the system. No need to block, since it's already finished.
        system = process.getSystem()

        return system


############# first minimise/equilibrate the solvated ligand.
print("\n#### Working on solvated ligand.")

print(f"Minimising in {minim_steps} steps..")
protocol = BSS.Protocol.Minimisation(steps=minim_steps)
minimised = runProcess(lig_p_solvated, protocol)

BSS.IO.saveMolecules(f"{tmp_dir}/min_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])


print(f"PMEMD NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_short_nvt*BSS.Units.Time.picosecond, 
                                temperature_start=0*BSS.Units.Temperature.kelvin, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                restraint="all"
                                )
equil1 = runProcess(minimised, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq1_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )

equil2 = runProcess(equil1, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq2_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
equil3 = runProcess(equil2, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq3_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )
lig_equil_fin = runProcess(equil3, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq4_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])


#### Working on solvated ligand.
Minimising in 250 steps..
PMEMD NVT equilibration for 5 ps while restraining all non-solvent atoms..
PMEMD NVT equilibration for 50 ps without restraints..
PMEMD NPT equilibration for 200 ps while restraining non-solvent heavy atoms..
PMEMD NPT equilibration for 200 ps without restraints..


['/home/2020/home/rory/project/ProjectDirectory/temp/eq4_free_lig_s.prm7',
 '/home/2020/home/rory/project/ProjectDirectory/temp/eq4_free_lig_s.rst7']

In [11]:
############# repeat for ligand + protein system. Include extra restrain="backbone" step.
print("\n#### Working on solvated ligand+protein.")
print(f"Minimising in {minim_steps} steps..")
protocol = BSS.Protocol.Minimisation(steps=minim_steps)
minimised = runProcess(system_solvated, protocol)


print(f"PMEMD NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_short_nvt*BSS.Units.Time.picosecond, 
                                temperature_start=0*BSS.Units.Temperature.kelvin, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                restraint="all"
                                )
equil1 = runProcess(minimised, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq1_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps while restraining all backbone atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature=300*BSS.Units.Temperature.kelvin, 
                                #restraint="backbone"
                                )
equil2 = runProcess(equil1, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq2_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                )

equil3 = runProcess(equil2, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq3_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
equil4 = runProcess(equil3, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq4_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )
sys_equil_fin = runProcess(equil4, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq5_mol_sys_s", system_solvated, ["PRM7", "RST7"])

# finally, save last snapshot of both equilibrated objects.
os.system("mkdir -p prep/ligands")
os.system("mkdir -p prep/protein")

print("Saving solvated/equilibrated systems.")
print("\n Ligand:")
print(lig_equil_fin)
BSS.IO.saveMolecules(f"prep_bases/ligands/free_lig_equil_solv", lig_equil_fin, ["PRM7", "RST7"])

print("\n Ligand + protein:")
print(sys_equil_fin)
BSS.IO.saveMolecules(f"prep_bases/protein/mol_sys_equil_solv", sys_equil_fin, ["PRM7", "RST7"])
print("First 20 molecules in ligand + protein system:")
for mol in sys_equil_fin.getMolecules()[:20]:
    print(mol)
print("Done.")


#### Working on solvated ligand+protein.
Minimising in 250 steps..
PMEMD NVT equilibration for 5 ps while restraining all non-solvent atoms..
PMEMD NVT equilibration for 50 ps while restraining all backbone atoms..
PMEMD NVT equilibration for 50 ps without restraints..
PMEMD NPT equilibration for 200 ps while restraining non-solvent heavy atoms..
PMEMD NPT equilibration for 200 ps without restraints..
Saving solvated/equilibrated systems.

 Ligand:
<BioSimSpace.System: nMolecules=2154>

 Ligand + protein:
<BioSimSpace.System: nMolecules=10000>
First 20 molecules in ligand + protein system:
<BioSimSpace.Molecule: number=26899, nAtoms=1062, nResidues=33>
<BioSimSpace.Molecule: number=36240, nAtoms=31, nResidues=1>
<BioSimSpace.Molecule: number=36241, nAtoms=3, nResidues=1>
<BioSimSpace.Molecule: number=36242, nAtoms=3, nResidues=1>
<BioSimSpace.Molecule: number=36243, nAtoms=3, nResidues=1>
<BioSimSpace.Molecule: number=36244, nAtoms=3, nResidues=1>
<BioSimSpace.Molecule: number=36245, 

## Objective 3b: Run MD simulation using OpenMM, pmemd.cuda and gromacs

To run a molecular dynamics simulation of the ligand (ligand_1) in the binding site of the RNA (rna_bases) we will make use of the BSS.Protocol and BSS.Process functions. 100 ns MD simulations of the ligand_1:rna_bases system  were run using AMBER, GROMACS and OpenMM. The MD simulations are initiated using the following [Python Script](/home/rory/project/ProjectDirectory/project_scripts_2/amber_run_bases.py) and queued using the following [Slurm Script](/home/rory/project/ProjectDirectory/project_scripts_2/amber_run_bases.sh). An example python script is shopwn below:

```python
import BioSimSpace as BSS
import os

print("Loading system...")
system= BSS.IO.readMolecules("../prep/protein/mol_sys_equil_solv.*")
protocol=BSS.Protocol.Production(runtime= 100 * BSS.Units.Time.nanosecond, report_interval=10_000)
process = BSS.Process.Amber(system, protocol,exe = "/usr/local/amber22/bin/pmemd.cuda", work_dir="output_amber")
process.start()
process.wait()


## Objective 3 Analysis: Plot RMSD and RMSF values for the trajectories.

RMSD Plots: for nucleic "nucleic", for backbone "nucleic or resid 13 or resid 14", for ligand "resname LIG", for all "nucleic or resid 13 or resid 14 or resname LIG".

In [ ]:
def rmsd_plot (topology, trajectory, selection, name):
    import MDAnalysis as mda
    from MDAnalysis.analysis import rms, align
    import matplotlib.pyplot as plt
    import numpy as np

    # Load the trajectory using DCDReader
    u = mda.Universe(topology, trajectory)

    # Select backbone atoms
    backbone_1 = selection

    # Calculate RMSD
    R = rms.RMSD(u, u, select= backbone_1)
    R.run()

    # Access RMSD data
    results = R.results.rmsd.T
    rmsds= results[2]
    time= results[1]
    fig, ax = plt.subplots()
    ax.plot(time, rmsds)
    ax.set_xlabel("Time / ps")
    ax.set_ylabel("RMSD / $\\AA$")
    ax.grid(True)
    plt.savefig(name)

rmsd_plot ('openmm.parm7', 'thinned_openmm.nc', 'nucleic', 'rmsd_openmm_nucleic')

The most useful analysis of the RNA system is to plot the RMSF values for the individual residues in the RNA backbone to determine the positions of flexibility in the system and hence determine any potential effects on the binding site. To do this we can use the following rmsf_plot function for all three simulations (AMBER, GROMACS and OpenMM):

In [ ]:
def rmsf_plot (topology_1, trajectory_1, selection_1, name_1):
     u_1 = mda.Universe(topology_1, trajectory_1)
     average = align.AverageStructure(u_1,u_1, select= selection_1, ref_frame=0).run()
     ref = average.results.universe

     aligner = align.AlignTraj(u_1, ref, select= selection_1, in_memory=True).run()
     rna = u_1.select_atoms(selection_1)
     R = rms.RMSF(rna).run()
     fig, ax = plt.subplots()
     ax.plot(R.rmsf)
     ax.set_xlabel("Relative Atom Index")
     ax.set_ylabel("RMSF / $\\AA$")
     tick_step = np.arange(0,1001, 100)
     ax.set_xticks(tick_step)
     plt.savefig(name)

rmsf_plot ('openmm.parm7', 'thinned_openmm.nc', 'nucleic', 'rmsf_openmm_nucleic')

## Objective 4: RBFE calculations for compounds 1-3 using SOMD and AMBER

## 4a: Set up of RBFE calculations.

In [ ]:
# import libraries
import BioSimSpace as BSS
import os
import glob
import csv
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

As ligand 1 is already in the binding site following the molecular dynamics simulations from Objective 3 it can be used to create merged molecules with ligands 2 and 3 respectively and complete the alchemical transformations necessary to compute Relative Binding Free Energy. 
- Ligand 1 in the binding site alchemically transformed into Ligand 2 
- Ligand 1 in the binding site alchemically transformed into Ligand 3

It is important to calculate the RBFE in both directions e.g.
- Ligand 1 in the binding site to Ligand 2 
- Ligand 2 in the binding site to Ligand 1 

As such we need to create ligand:rna systems for ligand 2 and lind 3 similar to ligand 1, and run minimisation and equilibration. There is no need to run MD simulations for ligand 2 and ligand 3 as the ligands are chemically similar to ligand 1

Below we have loaded the RNA system and Ligand 1 from the MD simulations along with Ligand 2, Ligand 3, and the parameterised waters. Ligands 2 and 3 required protonation prior to loading using obabel in the command line. An example of this is:

* obabel -i pdb lig_2.pdb -o pdb -O lig_2_protonated.pdb -p

In [ ]:

system = BSS.IO.readMolecules("../prep_bases/protein/mol_sys_equil_solv.*")
ligand_1_aligned = system[1]

pre_ligand_1 = BSS.IO.readMolecules("protonated_1.pdb")[0]
pre_ligand_2 = BSS.IO.readMolecules("protonated_2.pdb")[0]
pre_ligand_3 = BSS.IO.readMolecules("protonated_3.pdb")[0]
ligand_1 = BSS.Parameters.gaff2(pre_ligand_1).getMolecule()
ligand_2 = BSS.Parameters.gaff2(pre_ligand_2).getMolecule()
ligand_3 = BSS.Parameters.gaff2(pre_ligand_3).getMolecule()

waters = BSS.IO.readMolecules("../input_rory/waters.*")
rna_bases = system[0]
rna_bases_test = BSS.IO.readMolecules("/home/rory/project/ProjectDirectory/input_rory/input_rory_3q50/rna.*")

# Print the new variable
print('system =', system)

print ('rna =', rna_bases)
print ('ligand -',ligand_1)

To make sure Ligand 2 and Ligand 3 are in the binding sites of their respective Ligand:RNA systems we can align them to Ligand 1:

In [ ]:
mapping_3_1 = BSS.Align.matchAtoms(ligand_1_aligned, ligand_3)
inv_mapping = {v: k for k, v in mapping_3_1.items()}

# Align 3
ligand_3_aligned = BSS.Align.rmsdAlign(ligand_3, ligand_1_aligned, inv_mapping)

mapping_2_1 = BSS.Align.matchAtoms(ligand_1_aligned, ligand_2)
inv_mapping_2 = {v: k for k, v in mapping_2_1.items()}

#Align 2
ligand_2_aligned = BSS.Align.rmsdAlign(ligand_2, ligand_1_aligned, inv_mapping_2)

Now we can equilibrate and minimise Ligand 2 in the binding site.

In [ ]:
mol_2 = rna_bases + ligand_2_aligned + waters
lig_2 = ligand_2_aligned + waters 

#DETERMINE BOX SIZE AND SOLVATE COMBINED SYSTEM (CHECK COUNTER IONS)

box_min, box_max = rna_bases.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
padding = 15 * BSS.Units.Length.angstrom
box_length = max(box_size) + 2 * padding

molecules_sol = BSS.Solvent.tip3p(molecule= mol_2, box=3 * [box_length], ion_conc = 0.15)
search = molecules_sol.search("not mols with atomidx 2")
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")
    
    #DETERMINE BOX SIZE AND SOLVATE COMBINED SYSTEM (CHECK COUNTER IONS)

box_min, box_max = rna_bases.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
padding = 15 * BSS.Units.Length.angstrom
box_length = max(box_size) + 2 * padding

free_molecules_sol = BSS.Solvent.tip3p(molecule= lig_2, box=3 * [box_length], ion_conc = 0.15)
search = molecules_sol.search("not mols with atomidx 2")
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")
    
    ### Settings.
minim_steps = 250
runtime_short_nvt = 5 # ps
runtime_nvt = 50 # ps RESET to 50 after TESTING ! 
runtime_npt = 200 # ps RESET to 200 after TESTING !

### preamble. tmp_dir should at some point be derived using os.environ.get("")

tmp_dir = "temp_2"

amber_home = "/usr/local/amber22"
pmemd_path = amber_home + "/bin/pmemd.cuda" 

#################
lig_p_solvated = free_molecules_sol
system_solvated = molecules_sol
BSS.IO.saveMolecules(f"{tmp_dir}/free_lig_s", lig_p_solvated, ["PRM7", "RST7"])
BSS.IO.saveMolecules(f"{tmp_dir}/mol_sys_s", system_solvated, ["PRM7", "RST7"])
### Minimise and equilibrate our systems to make the ligand relax inside the pocket.

lig_p_solvated = BSS.IO.readMolecules([f"{tmp_dir}/free_lig_s.prm7", f"{tmp_dir}/free_lig_s.rst7"])
system_solvated = BSS.IO.readMolecules([f"{tmp_dir}/mol_sys_s.prm7", f"{tmp_dir}/mol_sys_s.rst7"])

print ("lig_p_solvated is =", lig_p_solvated)
print ("original is =", free_molecules_sol)


In [ ]:
def runProcess(system, protocol, pmemd=False):
        """
        Given a solvated system (BSS object) and BSS protocol, run a process workflow with either 
        Sander (CPU) or pmemd.cuda (GPU). NPT is typically done with GPU to save computing time.
        Returns the processed system.
        """
        # Create the process passing a working directory.
        if not pmemd:
            process = BSS.Process.Amber(system, protocol)
        elif pmemd:
            process = BSS.Process.Amber(system, protocol, exe=pmemd_path)

        # Start the process.
        process.start()

        # Wait for the process to exit.
        process.wait()

        # Check for errors.
        if process.isError():
            print(process.stdout())
            print(process.stderr())
            raise _Exceptions.ThirdPartyError("The process exited with an error!")

        # If it worked, try to get the system. No need to block, since it's already finished.
        system = process.getSystem()

        return system


############# first minimise/equilibrate the solvated ligand.
print("\n#### Working on solvated ligand.")

print(f"Minimising in {minim_steps} steps..")
protocol = BSS.Protocol.Minimisation(steps=minim_steps)
minimised = runProcess(lig_p_solvated, protocol)

BSS.IO.saveMolecules(f"{tmp_dir}/min_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])


print(f"PMEMD NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_short_nvt*BSS.Units.Time.picosecond, 
                                temperature_start=0*BSS.Units.Temperature.kelvin, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                restraint="all"
                                )
equil1 = runProcess(minimised, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq1_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )

equil2 = runProcess(equil1, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq2_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
equil3 = runProcess(equil2, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq3_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )
lig_equil_fin = runProcess(equil3, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq4_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

############# repeat for ligand + protein system. Include extra restrain="backbone" step.
print("\n#### Working on solvated ligand+protein.")
print(f"Minimising in {minim_steps} steps..")
protocol = BSS.Protocol.Minimisation(steps=minim_steps)
minimised = runProcess(system_solvated, protocol)


print(f"PMEMD NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_short_nvt*BSS.Units.Time.picosecond, 
                                temperature_start=0*BSS.Units.Temperature.kelvin, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                restraint="all"
                                )
equil1 = runProcess(minimised, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq1_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps while restraining all backbone atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature=300*BSS.Units.Temperature.kelvin, 
                                #restraint="backbone"
                                )
equil2 = runProcess(equil1, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq2_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                )

equil3 = runProcess(equil2, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq3_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
equil4 = runProcess(equil3, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq4_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )
sys_equil_fin = runProcess(equil4, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq5_mol_sys_s", system_solvated, ["PRM7", "RST7"])

# finally, save last snapshot of both equilibrated objects.
os.system("mkdir -p prep/ligands")
os.system("mkdir -p prep/protein")

print("Saving solvated/equilibrated systems.")
print("\n Ligand:")
print(lig_equil_fin)
BSS.IO.saveMolecules(f"prep_bases_2/ligands_2/free_lig_equil_solv", lig_equil_fin, ["PRM7", "RST7"])

print("\n Ligand + protein:")
print(sys_equil_fin)
BSS.IO.saveMolecules(f"prep_bases_2/protein_2/mol_sys_equil_solv", sys_equil_fin, ["PRM7", "RST7"])
print("First 20 molecules in ligand + protein system:")
for mol in sys_equil_fin.getMolecules()[:20]:
    print(mol)
print("Done.")

Now we can equilibrate and minimise Ligand 3 in the binding site.

In [ ]:
mol_2 = rna_bases + ligand_3_aligned + waters
lig_2 = ligand_3 + waters 

#DETERMINE BOX SIZE AND SOLVATE COMBINED SYSTEM (CHECK COUNTER IONS)

box_min, box_max = rna_bases.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
padding = 15 * BSS.Units.Length.angstrom
box_length = max(box_size) + 2 * padding

molecules_sol = BSS.Solvent.tip3p(molecule= mol_2, box=3 * [box_length], ion_conc = 0.15)
search = molecules_sol.search("not mols with atomidx 2")
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")
    
    #DETERMINE BOX SIZE AND SOLVATE COMBINED SYSTEM (CHECK COUNTER IONS)

box_min, box_max = rna_bases.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
padding = 15 * BSS.Units.Length.angstrom
box_length = max(box_size) + 2 * padding

free_molecules_sol = BSS.Solvent.tip3p(molecule= lig_2, box=3 * [box_length], ion_conc = 0.15)
search = molecules_sol.search("not mols with atomidx 2")
for ion in search.atoms():
    print(f"element = {ion.element()}, charge = {ion.charge()}")
    
    ### Settings.
minim_steps = 250
runtime_short_nvt = 5 # ps
runtime_nvt = 50 # ps RESET to 50 after TESTING ! 
runtime_npt = 200 # ps RESET to 200 after TESTING !

### preamble. tmp_dir should at some point be derived using os.environ.get("")

tmp_dir = "temp_3"

amber_home = "/usr/local/amber22"
pmemd_path = amber_home + "/bin/pmemd.cuda" 

#################
lig_p_solvated = free_molecules_sol
system_solvated = molecules_sol
BSS.IO.saveMolecules(f"{tmp_dir}/free_lig_s", lig_p_solvated, ["PRM7", "RST7"])
BSS.IO.saveMolecules(f"{tmp_dir}/mol_sys_s", system_solvated, ["PRM7", "RST7"])
### Minimise and equilibrate our systems to make the ligand relax inside the pocket.

lig_p_solvated = BSS.IO.readMolecules([f"{tmp_dir}/free_lig_s.prm7", f"{tmp_dir}/free_lig_s.rst7"])
system_solvated = BSS.IO.readMolecules([f"{tmp_dir}/mol_sys_s.prm7", f"{tmp_dir}/mol_sys_s.rst7"])

print ("lig_p_solvated is =", lig_p_solvated)
print ("original is =", free_molecules_sol)

In [ ]:
def runProcess(system, protocol, pmemd=False):
        """
        Given a solvated system (BSS object) and BSS protocol, run a process workflow with either 
        Sander (CPU) or pmemd.cuda (GPU). NPT is typically done with GPU to save computing time.
        Returns the processed system.
        """
        # Create the process passing a working directory.
        if not pmemd:
            process = BSS.Process.Amber(system, protocol)
        elif pmemd:
            process = BSS.Process.Amber(system, protocol, exe=pmemd_path)

        # Start the process.
        process.start()

        # Wait for the process to exit.
        process.wait()

        # Check for errors.
        if process.isError():
            print(process.stdout())
            print(process.stderr())
            raise _Exceptions.ThirdPartyError("The process exited with an error!")

        # If it worked, try to get the system. No need to block, since it's already finished.
        system = process.getSystem()

        return system


############# first minimise/equilibrate the solvated ligand.
print("\n#### Working on solvated ligand.")

print(f"Minimising in {minim_steps} steps..")
protocol = BSS.Protocol.Minimisation(steps=minim_steps)
minimised = runProcess(lig_p_solvated, protocol)

BSS.IO.saveMolecules(f"{tmp_dir}/min_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])


print(f"PMEMD NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_short_nvt*BSS.Units.Time.picosecond, 
                                temperature_start=0*BSS.Units.Temperature.kelvin, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                restraint="all"
                                )
equil1 = runProcess(minimised, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq1_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )

equil2 = runProcess(equil1, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq2_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
equil3 = runProcess(equil2, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq3_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )
lig_equil_fin = runProcess(equil3, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq4_free_lig_s", lig_p_solvated, ["PRM7", "RST7"])

############# repeat for ligand + protein system. Include extra restrain="backbone" step.
print("\n#### Working on solvated ligand+protein.")
print(f"Minimising in {minim_steps} steps..")
protocol = BSS.Protocol.Minimisation(steps=minim_steps)
minimised = runProcess(system_solvated, protocol)


print(f"PMEMD NVT equilibration for {runtime_short_nvt} ps while restraining all non-solvent atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_short_nvt*BSS.Units.Time.picosecond, 
                                temperature_start=0*BSS.Units.Temperature.kelvin, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                restraint="all"
                                )
equil1 = runProcess(minimised, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq1_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps while restraining all backbone atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature=300*BSS.Units.Temperature.kelvin, 
                                #restraint="backbone"
                                )
equil2 = runProcess(equil1, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq2_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NVT equilibration for {runtime_nvt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_nvt*BSS.Units.Time.picosecond, 
                                temperature_end=300*BSS.Units.Temperature.kelvin,
                                )

equil3 = runProcess(equil2, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq3_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps while restraining non-solvent heavy atoms..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                restraint="heavy",
                                )
equil4 = runProcess(equil3, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq4_mol_sys_s", system_solvated, ["PRM7", "RST7"])

print(f"PMEMD NPT equilibration for {runtime_npt} ps without restraints..")
protocol = BSS.Protocol.Equilibration(
                                runtime=runtime_npt*BSS.Units.Time.picosecond, 
                                pressure=1*BSS.Units.Pressure.atm,
                                temperature=300*BSS.Units.Temperature.kelvin,
                                )
sys_equil_fin = runProcess(equil4, protocol, pmemd=True)

BSS.IO.saveMolecules(f"{tmp_dir}/eq5_mol_sys_s", system_solvated, ["PRM7", "RST7"])

# finally, save last snapshot of both equilibrated objects.
os.system("mkdir -p prep/ligands")
os.system("mkdir -p prep/protein")

print("Saving solvated/equilibrated systems.")
print("\n Ligand:")
print(lig_equil_fin)
BSS.IO.saveMolecules(f"prep_bases_3/ligands_3/free_lig_equil_solv", lig_equil_fin, ["PRM7", "RST7"])

print("\n Ligand + protein:")
print(sys_equil_fin)
BSS.IO.saveMolecules(f"prep_bases_3/protein_3/mol_sys_equil_solv", sys_equil_fin, ["PRM7", "RST7"])
print("First 20 molecules in ligand + protein system:")
for mol in sys_equil_fin.getMolecules()[:20]:
    print(mol)
print("Done.")

## Create Merged Molecules

We will start with ligand 1 in the binding site. Create merged molecules between ligands 1 and 2, and 1 and 3:

In [ ]:
# create the mapping and the inverse mapping
mapping_3_1 = BSS.Align.matchAtoms(ligand_1_aligned, ligand_3)
inv_mapping = {v: k for k, v in mapping_3_1.items()}

# Align
ligand_3_1_aligned = BSS.Align.rmsdAlign(ligand_3, ligand_1_aligned, inv_mapping)

# Merge the two ligands based on the mapping.
merged_3_1_aligned = BSS.Align.merge(ligand_1_aligned, ligand_3_1_aligned, mapping_3_1)

#Create complex and free merged system

complx_3_1 = merged_3_1_aligned + rna_bases 

merged_3_1 = merged_3_1_aligned 

#Solvate

complx_3_1_sol = BSS.Solvent.tip3p(molecule= complx_3_1, box=3 * [box_length], ion_conc = 0.15)

merged_3_1_sol = BSS.Solvent.tip3p(molecule= merged_3_1, box=3 * [box_length], ion_conc = 0.15)
# create the mapping and the inverse mapping
mapping_2_1 = BSS.Align.matchAtoms(ligand_1_aligned, ligand_2)
inv_mapping = {v: k for k, v in mapping_2_1.items()}

# Align
ligand_2_1_aligned = BSS.Align.rmsdAlign(ligand_2, ligand_1_aligned, inv_mapping)

# Merge the two ligands based on the mapping.
merged_2_1_aligned = BSS.Align.merge(ligand_1_aligned, ligand_2_1_aligned, mapping_2_1)

#Create complex and free merged system

complx_2_1 = merged_2_1_aligned + rna_bases

merged_2_1 = merged_2_1_aligned

#solvate

complx_2_1_sol = BSS.Solvent.tip3p(molecule= complx_2_1, box=3 * [box_length], ion_conc = 0.15)

merged_2_1_sol = BSS.Solvent.tip3p(molecule= merged_2_1, box=3 * [box_length], ion_conc = 0.15)

It is helpful to view the merged molecules to see that they are in the binding site and have hence aligned to the correct ligand.

In [ ]:
rna_test = complx_2_1
view = BSS.Notebook.View(rna_test)
view.system()

Now we can create merged moleucles with Ligand 2 in the binding site. Create merged molecules between ligands 2 and 3, and 2 and 1:

In [ ]:
# create the mapping and the inverse mapping
mapping_3_2 = BSS.Align.matchAtoms(ligand_2, ligand_3)
inv_mapping = {v: k for k, v in mapping_3_2.items()}

# Align
ligand_3_2_aligned = BSS.Align.rmsdAlign(ligand_3, ligand_2, inv_mapping)

# Merge the two ligands based on the mapping.
merged_3_2_aligned = BSS.Align.merge(ligand_2, ligand_3_2_aligned, mapping_3_2)

#Create complex and free merged system

complx_3_2 = merged_3_2_aligned + rna_bases_2 

merged_3_2 = merged_3_2_aligned 

#Solvate

complx_3_2_sol = BSS.Solvent.tip3p(molecule= complx_3_2, box=3 * [box_length], ion_conc = 0.15)

merged_3_2_sol = BSS.Solvent.tip3p(molecule= merged_3_2, box=3 * [box_length], ion_conc = 0.15)

# create the mapping and the inverse mapping
mapping_1_2 = BSS.Align.matchAtoms(ligand_2, ligand_1)
inv_mapping = {v: k for k, v in mapping_1_2.items()}

# Align
ligand_1_2_aligned = BSS.Align.rmsdAlign(ligand_1, ligand_2, inv_mapping)

# Merge the two ligands based on the mapping.
merged_1_2_aligned = BSS.Align.merge(ligand_2, ligand_1_2_aligned, mapping_1_2)

#Create complex and free merged system

complx_1_2 = merged_1_2_aligned + rna_bases_2 

merged_1_2 = merged_1_2_aligned 

#Solvate

complx_1_2_sol = BSS.Solvent.tip3p(molecule= complx_1_2, box=3 * [box_length], ion_conc = 0.15)

merged_1_2_sol = BSS.Solvent.tip3p(molecule= merged_1_2, box=3 * [box_length], ion_conc = 0.15)



Now we can create merged moleucles with Ligand 3 in the binding site. Create merged molecules between ligands 3 and 2, and 3 and 1:

In [ ]:
# create the mapping and the inverse mapping
mapping_2_3 = BSS.Align.matchAtoms(ligand_2, ligand_3)
inv_mapping_2_3 = {v: k for k, v in mapping_2_3.items()}

# Align
ligand_2_3_aligned = BSS.Align.rmsdAlign(ligand_3, ligand_2, inv_mapping_2_3)

# Merge the two ligands based on the mapping.
merged_2_3_aligned = BSS.Align.merge(ligand_2, ligand_2_3_aligned, mapping_2_3)

#Create complex and free merged system

complx_2_3 = merged_2_3_aligned + rna_bases_3 

merged_2_3 = merged_2_3_aligned 

#solvate

complx_2_3_sol = BSS.Solvent.tip3p(molecule= complx_2_3, box=3 * [box_length], ion_conc = 0.15)

merged_2_3_sol = BSS.Solvent.tip3p(molecule= merged_2_3, box=3 * [box_length], ion_conc = 0.15)

# create the mapping and the inverse mapping
mapping_1_3 = BSS.Align.matchAtoms(ligand_1, ligand_3)
inv_mapping_1_3 = {v: k for k, v in mapping_1_3.items()}

# Align
ligand_1_3_aligned = BSS.Align.rmsdAlign(ligand_3, ligand_1, inv_mapping_1_3)

# Merge the two ligands based on the mapping.
merged_1_3_aligned = BSS.Align.merge(ligand_1, ligand_1_3_aligned, mapping_1_3)

#Create complex and free merged system

complx_1_3 = merged_1_3_aligned + rna_bases_3 

merged_1_3 = merged_1_3_aligned 

#solvate

complx_1_3_sol = BSS.Solvent.tip3p(molecule= complx_1_3, box=3 * [box_length], ion_conc = 0.15)

merged_1_3_sol = BSS.Solvent.tip3p(molecule= merged_1_3, box=3 * [box_length], ion_conc = 0.15)

## Generate .pert files ready for RBFE calculations

Now that we have merged molecules for Ligand 1:Ligand 2 and Ligand 1:Ligand 3 we can generate the input files (.pert) for the RBFE calculations using BSS.FreeEnergy.Relative.

- Note: in the extra options directory I have specified nmoves = 25000, ncycles = 50, timestep = 4 fs and hydrogen mass repartitioning factor = 3.0. This results in 5 ns simulation time and is done to ensure full engagement of GPUs.
- Currently the lamba window configuration files require 'femtosecond' rather than 'fs'. NO FIX CURRENTLY

In [ ]:
def pert_gen(ligand, system, name)
    ### Settings.
    minim_steps = 250
    runtime_short_nvt = 5 # ps
    runtime_nvt = 50 # ps RESET to 50 after TESTING ! 
    runtime_npt = 200 # ps RESET to 200 after TESTING !

    ### preamble. tmp_dir should at some point be derived using os.environ.get("")

    tmp_dir = "temp"

    amber_home = "/usr/local/amber22"
    pmemd_path = amber_home + "/bin/pmemd.cuda" 

    #################
    lig_p_solvated = ligand
    system_solvated = system

    print ("lig_p_solvated is =", lig_p_solvated)
    print ("original is =", ligand)

    freenrg_protocol = BSS.Protocol.FreeEnergy(
        num_lam=16, runtime= 5 * BSS.Units.Time.nanosecond
    )
    # set up a bound folder with standard settings.
    # Use solvation to prep the bound leg
    print("Bound..")
    workdir = f"prep_bases/rbfe_outputs/{name}"
    BSS.FreeEnergy.Relative(
        system_solvated,
        freenrg_protocol,
        engine=f"SOMD",
        work_dir=workdir + "/bound",
        extra_options={ "nmoves" : 25000, "ncycles" : 50, "timestep" : 4 * BSS.Units.Time.femtosecond, "hydrogen mass repartitioning factor" : 3.0},
    )

    # set up a free folder.
    print("Free..")
    BSS.FreeEnergy.Relative(
        lig_p_solvated,
        freenrg_protocol,
        engine=f"SOMD",
        work_dir=workdir + "/free",
        extra_options={ "nmoves" : 25000, "ncycles" : 50, "timestep" : 4 * BSS.Units.Time.femtosecond, "hydrogen mass repartitioning factor" : 3.0},
    )
pert_gen(merged_2_1_sol, complx_2_1_sol, "lig_2_1")

# 4b. Analyse RBFE

Calculate the RBFE and compare to a literature value.

In [ ]:
pmf_free, overlap_matrix_free = BSS.FreeEnergy.Relative.analyse(f'prep_bases/rbfe_outputs/lig_1_2_r1/free')
pmf_bound, overlap_matrix_bound = BSS.FreeEnergy.Relative.analyse(f'prep_bases/rbfe_outputs/lig_1_2_r1/bound')
freenrg_rel = BSS.FreeEnergy.Relative.difference(pmf_bound, pmf_free)

print(f"pmf_free is: {pmf_free[-1][1]} and the MBAR statistical uncertainty is {pmf_free[-1][2]} .")
print(f"pmf_bound is: {pmf_bound[-1][1]} and the MBAR statistical uncertainty is {pmf_bound[-1][2]} .")
print(f"The RBFE is {freenrg_rel[0]} and the error is {freenrg_rel[1]}")

Below is a function to calculate the RBFE between two literature ligands and convert the value to kcal/mol.

In [ ]:
import math

def kd_to_deltaG(kd, temperature):
    # Gas constant in J/(mol·K)
    R = 8.314

    # Convert temperature to Kelvin if it's given in Celsius
    if temperature < 273.15:
        temperature += 273.15

    # Calculate delta G in joules/mol
    delta_G_joules = -R * temperature * math.log(kd)

    # Convert delta G to kilocalories/mol
    delta_G_kcal = delta_G_joules * 0.000239

    return delta_G_kcal

# Example usage
kd_value = 0.5 * 1e-6  # Kd in micro M
temperature_value = 300  # Temperature in Kelvin

deltaG_result = kd_to_deltaG(kd_value, temperature_value)
print(f"Free Energy (ΔG) of 1 = {deltaG_result} kcal/mol")

kd_value_2 = 0.4 * 1e-6  # Kd in micro M
temperature_value_2 = 300  # Temperature in Kelvin

deltaG_result_2 = kd_to_deltaG(kd_value_2, temperature_value_2)
print(f"Free Energy (ΔG) of 2 = {deltaG_result_2} kcal/mol")

diff = deltaG_result_2 - deltaG_result
print (f"diff = {diff} kcal/mol")